# Commant interpretation experimnets
This notebook contains the experiments necessary for selecting and calling a command

## Command filtering
There are a lot of commands, and we need to filter them, as the next step that directly uses functions (function calling) costs much less if we only use the top 5 recommended commands, for the cost of a very little slice of punctuality

In [1]:
import pandas as pd

df = pd.read_csv('misc/commands.csv', header=0, sep=";")
df

C:\Users\szcsa\AppData\Local\Temp\ipykernel_1056\3514612687.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,Name,Example,Description,Param1,Param1Type,Param1Required,Param1Description,Param2,Param2Type,Param2Required,Param2Description,Alternative
0,set_alarm,set an alarm for 10 oclock,Sets an alarm for the given time,time,string,1.0,The given time in HH:MM 24H format,message,string,0.0,the message or the name of the alarm,0
1,set_timer,set a timer for 10 minutes,Sets a timer for a given period,time,string,1.0,The given time period in HH:MM:SS format,message,string,0.0,the message or the name of the reminder,0
2,take_picture,take a picture or open camera,opens camera application in picture mode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,record_video,record a video,opens camera application in video mode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,search_web,search for the term something on the internet ...,Searches the web with the given query parameter,query,string,1.0,The thing to search on the internet,NaN,NaN,NaN,NaN,0
5,bluetooth_on,turn on bluetooth,turns on bluetooth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,settings,open the settings,Opens general settings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,wifi_settings,open wifi settings,Opens wifi settings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,wireless_settings,open wireless settings,Opens wireless settings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
9,airplane_settings,open airplane mode settings,Opens airplane mode settings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

#HF embeddings
embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")

In [50]:
# two env variables needed: AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT

import os

os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["AZURE_OPENAI_ENDPOINT"] = ""

In [5]:
from langchain_openai import AzureOpenAIEmbeddings

#openai embeddings
#embeddings = AzureOpenAIEmbeddings(model="text-embedding-ada-002")

embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-large-test")

#embeddings = AzureOpenAIEmbeddings(model="text-embedding-3-small-test") #error

In [21]:
from langchain_core.documents import Document

docs = []

for index, row in df.iterrows():
    doc = Document(
        page_content=row['Description'],
        metadata={
            "source": row['Name'], 
            "param1": row['Param1'], "param1type": row['Param1Type'], "param1required": row['Param1Required'], "param1description": row['Param1Description'],
            "param2": row['Param2'], "param2type": row['Param2Type'], "param2required": row['Param2Required'], "param2description": row['Param2Description']
        },
    )
    docs.append(doc)


In [22]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

In [23]:
db

In [24]:
functions = db.similarity_search(
    "nyisd meg a dátum beállításokat",
    k=5,
)

functions

[Document(metadata={'source': 'date_settings', 'param1': nan, 'param1type': nan, 'param1required': nan, 'param1description': nan, 'param2': nan, 'param2type': nan, 'param2required': nan, 'param2description': nan}, page_content='opens date settings'),
 Document(metadata={'source': 'add_event', 'param1': 'title', 'param1type': 'string', 'param1required': 0.0, 'param1description': 'The title for the calendar event', 'param2': 'date', 'param2type': 'string', 'param2required': 1.0, 'param2description': 'the start date of the event in MM.dd. format'}, page_content='Adds an event on the name title to the calendar'),
 Document(metadata={'source': 'set_timer', 'param1': 'time', 'param1type': 'string', 'param1required': 1.0, 'param1description': 'The given time period in HH:MM:SS format', 'param2': 'message', 'param2type': 'string', 'param2required': 0.0, 'param2description': 'the message or the name of the reminder'}, page_content='Sets a timer for a given period'),
 Document(metadata={'source'

## Integrating filtering into function calling
The result of filtering should be parsed into a function call to determine the correct function

In [13]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-06-01",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [14]:
llm.invoke("hello!") 

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_80a1bad4c7', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-1eacdd16-5135

In [36]:
tool_list = []

# according to https://blog.langchain.dev/tool-calling-with-langchain/

for function in functions:

    # isinstance to check type, as nan is not a string

    # has 2 params
    if isinstance(function.metadata["param2"], str):
        required = []
        if (function.metadata["param1required"] == 1.0):
            required.append(function.metadata["param1"])

        if (function.metadata["param2required"] == 1.0):
            required.append(function.metadata["param2"])
        
        tool = {
            "name": function.metadata['source'],
            "description": function.page_content,
            "parameters" : {
                "type": "object",
                "properties": {
                    function.metadata["param1"]: {"type": function.metadata["param1type"], "description": function.metadata["param1description"]},
                    function.metadata["param2"]: {"type": function.metadata["param2type"], "description": function.metadata["param2description"]},
                },
            },
            "required": required
        }
        tool_list.append(tool)

    # has 1 param
    elif isinstance(function.metadata["param1"], str):
        required = []
        if (function.metadata["param1required"] == 1.0):
            required.append(function.metadata["param1"])
        
        tool = {
            "name": function.metadata['source'],
            "description": function.page_content,
            "parameters" : {
                "type": "object",
                "properties": {
                  function.metadata["param1"]: {"type": function.metadata["param1type"], "description": function.metadata["param1description"]}
                }
            },
            "required": required
        }
        tool_list.append(tool)

    #no param
    else:
        tool = {
            "name": function.metadata['source'],
            "description": function.page_content,
            "parameters" : {},
            "required": []
        }
        tool_list.append(tool)


standard else path
param2 add_event
param2 set_timer
param2 set_alarm
standard else path


In [38]:
llm_with_tools = llm.bind_tools(tool_list)

messages = [
    (
        "system",
        "You are a helpful assistant that decides to choose the correct function from the user prompt, or choose none of them if none of them match.",
    ),
    ("human", "nyisd meg a dátum beállításokat"),
]
ai_msg = llm_with_tools.invoke(messages)
ai_msg.tool_calls[0]['name']

'date_settings'

## Connecting the whole flow

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS


class CommandResolver:
    """
    Class to choose the appropriate runnable command.
    """

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                ("You are a helpful assistant that decides which function should be choosed based on the user prompt."
                "If no function is close to the user prompt, then simply do not select a function")
            ),
            ("human", "{user_prompt}"),
        ]
    )

    df = pd.read_csv('misc/commands.csv', header=0, sep=";")

    embeddings = HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")

    docs = []

    for index, row in df.iterrows():
        doc = Document(
            page_content=row['Description'],
            metadata={
                "source": row['Name'], 
                "param1": row['Param1'], "param1type": row['Param1Type'], "param1required": row['Param1Required'], "param1description": row['Param1Description'],
                "param2": row['Param2'], "param2type": row['Param2Type'], "param2required": row['Param2Required'], "param2description": row['Param2Description']
            },
        )
        docs.append(doc)

    db = FAISS.from_documents(docs, embeddings)

    llm = AzureChatOpenAI(
        azure_deployment="gpt-4o",
        api_version="2024-06-01",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    @classmethod
    async def select(cls, text: str) -> Tuple[dict,str]:
        """Selects the correct function based on the user prompt.

        Args:
            text (str): The user prompt.

        Returns:
            Tuple[dict,str]: The selected function and an error answer
        """

        if text.strip() == "":
            return False, ""     

        functions = cls.db.similarity_search(
            text,
            k=5,
        )
    
        tool_list = []
        
        # according to https://blog.langchain.dev/tool-calling-with-langchain/
        
        for function in functions:
        
            # isinstance to check type, as nan is not a string
        
            # has 2 params
            if isinstance(function.metadata["param2"], str):
                required = []
                if (function.metadata["param1required"] == 1.0):
                    required.append(function.metadata["param1"])
        
                if (function.metadata["param2required"] == 1.0):
                    required.append(function.metadata["param2"])
                
                tool = {
                    "name": function.metadata['source'],
                    "description": function.page_content,
                    "parameters" : {
                        "type": "object",
                        "properties": {
                            function.metadata["param1"]: {"type": function.metadata["param1type"], "description": function.metadata["param1description"]},
                            function.metadata["param2"]: {"type": function.metadata["param2type"], "description": function.metadata["param2description"]},
                        },
                    },
                    "required": required
                }
                tool_list.append(tool)
        
            # has 1 param
            elif isinstance(function.metadata["param1"], str):
                required = []
                if (function.metadata["param1required"] == 1.0):
                    required.append(function.metadata["param1"])
                
                tool = {
                    "name": function.metadata['source'],
                    "description": function.page_content,
                    "parameters" : {
                        "type": "object",
                        "properties": {
                          function.metadata["param1"]: {"type": function.metadata["param1type"], "description": function.metadata["param1description"]}
                        }
                    },
                    "required": required
                }
                tool_list.append(tool)
        
            #no param
            else:
                tool = {
                    "name": function.metadata['source'],
                    "description": function.page_content,
                    "parameters" : {},
                    "required": []
                }
                tool_list.append(tool)
        
        llm_with_funcs = cls.llm.bind_tools(tool_list)

        chain = cls.prompt | llm_with_funcs
   
        try:
            res = await chain.ainvoke({"user_prompt": text})        
            if hasattr(res, 'tool_calls') and len(res.tool_calls) > 0:
                
                called_function_name = res.tool_calls[0]['name']
                called_function_desc = [function for function in tool_list if function['name'] == called_function_name][0]['description']

                prompt = ChatPromptTemplate.from_messages(
                    [
                        (
                            "system",
                            ("You are a friendly assistant, and you should give a polite short answer to the user prompt as a feedback that it will do an action."
                            f"The called function is {called_function_name}, and its description is {called_function_desc}.")
                        ),
                        ("human", "{user_prompt}"),
                    ]
                )
                chain_react = prompt | cls.llm
                answer = await chain_react.ainvoke({"user_prompt": text})

                return res.tool_calls[0], answer.content
                
            else:
                return {}, res.content

        except Exception as e:
            raise Exception(e) 

print(await CommandResolver.select("set a timer for 10 minutes"))

C:\Users\szcsa\miniconda3\envs\voice_assistant\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


({'name': 'set_timer', 'args': {'time': '00:10:00'}, 'id': 'call_DT5WidcTSUXP1rpfDOXdkRRt', 'type': 'tool_call'}, "Sure, I'll set a timer for 10 minutes.")


## Agent for decision
Deciding if a user prompt is a question or an action is an important task, as it vastly changes the processes

In [15]:
from typing import Tuple

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI


class DecisionAgent:
    """
    Class to decide if a user prompt is a question or a runnable command.
    """

    class TextClassification(BaseModel):
        """
        Classifies the user prompt if it is a quesion or a runnable command
        """
        runnable: bool = Field(
            description=(
                "'True' if the text is a runnable or executable command, even if it is given in a question-like form."
                "'False' if the text is a regular question or a question about the possibility of something."
            )
        )

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a friendly assistant. Classify the user prompt if it is a quesion or a runnable command.",
            ),
            ("human", "{user_prompt}"),
        ]
    )

    llm = AzureChatOpenAI(
        azure_deployment="gpt-4o", #or mini or normal
        api_version="2024-06-01",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    ).with_structured_output(TextClassification)

    chain = prompt | llm

    @classmethod
    async def decide(cls, text: str) -> bool:
        """Decide if the user prompt is runnable or not.

        Args:
            text (str): The piece of text to analyze.

        Returns:
            bool: Indicates wheter the user prompt wants to run something or it is just a question
        """

        if text.strip() == "":
            return False       
        
        try:
            res = await cls.chain.ainvoke({"user_prompt": text})
            return res.runnable

        except Exception as e:
            raise Exception(e) 

print(await DecisionAgent.decide("Nyisd meg a beállításokat"))

True


# Test Framework for Command Interpretation

In [43]:
tests = [#(prompt, expected_command),
    ("állíts be egy 5 perces időzítőt", "set_timer")
        ]

In [48]:
import csv

async def process_prompts(resolver, tests, path=""):

    predicted_command_list = []
    answer_list = []
    
    passed_count = 0
    test_count = len(tests)

    
    for prompt, expected_command in tests:
        print(prompt)
        predicted_command, answer = await resolver.select(prompt)
        predicted_command_name = predicted_command["name"]
        predicted_command_list.append(predicted_command_name)
        answer_list.append(answer)

        if predicted_command_name == expected_command:
            passed_count += 1

    print("Passed tests: ", passed_count)
    print("Passed ratio: ", passed_count/test_count)

    # Create a CSV file and write the data
    with open(f'test_results_{path}.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Prompt', 'Expected command', 'Actual command', 'Answer'])

        for i in range(test_count):
            writer.writerow([tests[i][0], tests[i][1], predicted_command_list[i], answer_list[i]])


In [49]:
cr = CommandResolver()

await process_prompts(cr, tests)

állíts be egy 5 perces időzítőt
Passed tests:  1
Passed ratio:  1.0


# Test Framework for Decision Agent

In [45]:
tests_for_decision = [#(prompt, is_it_runnable),
    ("Nyisd meg a beállításokat", True),
    ("Mit vegyek fel 16 fokban", False),
    
        ]
len(tests_for_decision)

2

In [46]:
da = DecisionAgent()

async def process_decision_prompts(decision_agent, tests, path=""):

    predicted_result_list = []
    
    passed_count = 0
    test_count = len(tests)

    
    for prompt, expected_result in tests:
        predicted_result = await decision_agent.decide(prompt)
        predicted_result_list.append(predicted_result)

        if predicted_result == expected_result:
            passed_count += 1

    print("Passed tests: ", passed_count)
    print("Passed ratio: ", passed_count/test_count)

    # Create a CSV file and write the data
    with open(f'decision_test_results_{path}.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Prompt', 'Expected result', 'Actual result'])

        for i in range(test_count):
            writer.writerow([tests[i][0], tests[i][1], predicted_result_list[i]])

await process_decision_prompts(da, tests_for_decision)

Passed tests:  2
Passed ratio:  1.0


# Answer Agent for natural language questions

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import AzureChatOpenAI


class AnswerAgent:
    """
    Class to answer a user prompt.
    """
    
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a friendly voice assistant. Answer the question on the source language with less than 3 sentences",
            ),
            ("human", "{user_prompt}"),
        ]
    )

    llm = AzureChatOpenAI(
        azure_deployment="gpt-4o", #or mini or normal
        api_version="2024-06-01",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
    )

    chain = prompt | llm

    @classmethod
    async def answer(cls, text: str) -> str:
        """Answer the user prompt.

        Args:
            text (str): The piece of text to answer.

        Returns:
            str: The answer for the question
        """

        if text.strip() == "":
            return False       
        
        try:
            res = await cls.chain.ainvoke({"user_prompt": text})
            return res.content

        except Exception as e:
            raise Exception(e) 

print(await AnswerAgent.answer("Milyen ruhát vegyek fel 12 fokban?"))

12 fokban érdemes rétegesen öltözködni: viselj egy hosszú ujjú felsőt, egy könnyű pulóvert vagy kardigánt, és egy közepesen vastag kabátot. Ne felejtsd el a kényelmes nadrágot és a zárt cipőt sem!
